In [16]:
# En esta celda se puede probar la funcion de analisis de una imagen

from lib.image import image_data
from lib.analysis import dyssynchrony_analysis
import cv2

# Path completo donde esta la imagen (incluyendo nombre y extencion)
path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/seleccion/050320sel/050320c1/'
photo_name = '1b000.tif'
# Ancho de la feta a analisar (es un parametro opcional)
slice_width = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 70

# Calibracion del tiempo de cada pixel
calibration = 3.1

image = cv2.imread(path + photo_name)
# Select ROI
fromCenter = False
showCrosshair = False
seleted_parameters = cv2.selectROI(image, fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.destroyAllWindows()

image = image_data.get_image_data(image)
image = image_data.crop_vertical(image, x_start, x_end)
image = image_data.crop_horizontal(image, y_start, y_end)

results = dyssynchrony_analysis.analyze_image(image, min_dist_between_maxs, calibration, slice_width=slice_width)

# PRUEBA ANALISIS 

In [ ]:
# En esta celda se puede probar la funcion de analisis todas las imagenes .tif en un directorio
import os
from lib.image import image_data
from lib.analysis import dyssynchrony_analysis
import cv2

# Ancho de la feta a analisar (es un parametro opcional)
slice_width = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 200

# Calibracion del tiempo de cada pixel
calibration = 3.1

# Path donde estan las imagenes
path = "C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/1 Hz/"

images_paths = [ f'{path}/{file}' for file in os.listdir(path) if file.endswith(".tif") ]
images = [ cv2.imread(image_path) for image_path in images_paths ]

In [ ]:
fromCenter = False
showCrosshair = False

#Tomamos la primer imagen para seleccionar el recorte elegido el resto de las imagenes se van a recortar igual
seleted_parameters = cv2.selectROI(images[0], fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.destroyAllWindows()

In [ ]:
images = [ image_data.get_image_data(image) for image in images ]
images = [ image_data.crop_vertical(image, x_start, x_end) for image in images ]
images = [ image_data.crop_horizontal(image, y_start, y_end) for image in images ]

In [ ]:
results = [(dyssynchrony_analysis.analyze_image(image, min_dist_between_maxs, calibration, slice_width=5), print ('Done')) for image in images]

# Analisis de datos
Notas: 
- Image corresponde al análisis de toda la célula y slices corresponde a fetas de esa imagen
- intensidades es el valor que resulta de comprimir (sumando) la matriz con las coordenadas de x,y,z de los pixeles 
- max_peaks_pos es la index de en la lista de intensidades 
- max_peaks_intensities el valor de intensidad que se corresponde con el pico en la posicion analoga de max_peaks_pos

In [3]:
#Imports
import json
import numpy as np
import pandas as pd

In [4]:
#encodes a dictionary into a jason
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def jasonGenerator(path,results):
    """ Generates a .jason file in the path given folder from the a dictionary """
    with open(path + '/analysis_result.json',"w") as miarch:
        miarch.write(json.dumps(results, cls=NumpyEncoder))

In [5]:
def jasonReader(path):
    """ Reads the .jason generated with jasonGenartor the given path"""
    with open( path + '/analysis_result.json','r') as miarch:
        loaded_dict = json.loads(miarch.read())
    return loaded_dict

In [6]:
def slicesParser_max_peaksI(path):
#     read the analysis result 
    dictres = jasonReader(path)

#extract all the slices data
    allSlices = dictres['slices']
    try:
        #seting columns names
        column_names = ['transient' + str(x) for x in range(1,len(allSlices[0]['max_peaks_intensities']))]    
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        for i in range(0,len(allSlices)):
            df_sum.loc[i] = allSlices[i]['max_peaks_intensities'][1:] 
        return df_sum

    except:
        print('no possible callculation')

In [7]:
def slicesParser_halfpeakstime(path):
    #read the analysis result 
    dictres = jasonReader(path)
    #extract all the slices data
    allSlices = dictres['slices']
#     try:
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_half_peaks']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['times_to_half_peaks']
    return df_sum

#     except:
#         print('no possible callculation')

In [8]:
def slicesParser_amplitudes(path):
    #read the analysis result 
    dictres = jasonReader(path)
    #extract all the slices data
    allSlices = dictres['slices']
    try:
        #seting columns names
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['amplitudes']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['amplitudes']

        return df_sum
    
    except:
        print('no possible callculation')

In [9]:
def slicesParser_min_peaksI(path):
    #read the analysis result 
    dictres = jasonReader(path)
    #extract all the slices data
    allSlices = dictres['slices']
    try:
        #seting columns names
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['min_peaks_intensities']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['min_peaks_intensities']

        return df_sum
    
    except:
        print('no possible callculation')

In [10]:
def slicesParser_peaktime(path):
    #read the analysis result 
    dictres = jasonReader(path)
    #extract all the slices data
    allSlices = dictres['slices']

    try:
        #seting columns names
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_peaks']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['times_to_peaks']

        return df_sum
    
    except:
        print('no possible callculation')

In [11]:
def slicesParser_tau(path):
    #read the analysis result 
    dictres = jasonReader(path)
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    try:
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['tau_s']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['tau_s']
        
        return df_sum
    
    except:
        print('no possible callculation')

In [12]:
def BasicStasts(df):
    if df.empty == False:
        return df.astype('int').describe()

In [13]:
def discordIndex(df):
    return df.iloc[2].mean()

In [14]:
def amplitudes_ratio(df):
    df_alt = pd.DataFrame()
    for i in range(0,len(list(df))-1):
        name = 'ratio' + str(i+1) +'-' + str(i)
        df_alt[name] = df[list(df)[i+1]]/df[list(df)[i]]
    return df_alt

In [16]:
jasonGenerator(path,results)

In [17]:
loaded_dict = jasonReader(path)

In [18]:
BasicStasts(slicesParser_max_peaksI(path))

,transient1,transient2,transient3,transient4,transient5
count,38.000000,38.000000,38.000000,38.000000,38.000000
mean,24.263158,24.236842,24.000000,23.842105,23.631579
std,1.266709,1.324085,1.414214,1.385764,1.100885
min,23.000000,23.000000,22.000000,22.000000,22.000000
25%,23.000000,23.000000,23.000000,23.000000,23.000000
50%,24.000000,24.000000,24.000000,23.500000,23.000000
75%,25.000000,25.000000,25.000000,24.750000,24.000000
max,28.000000,28.000000,28.000000,28.000000,27.000000


In [23]:
amplitudes_ratio(slicesParser_amplitudes(path)).head(2)

,ratio1-0,ratio2-1,ratio3-2,ratio4-3
1,1.181818,1.0,0.916667,0.839161
2,1.000000,1.0,1.181818,0.769231


In [21]:
slicesParser_min_peaksI(path).head(2)

,transient1,transient2,transient3,transient4,transient5
1,13,12,12,12,13
2,13,13,13,12,13


In [24]:
slicesParser_tau(path).head(2)

,transient1,transient2,transient3,transient4,transient5
1,3.726676,4.595407,5.504581,6.903166,6.950928
2,3.705613,4.565008,5.436329,6.885360,7.167784


In [27]:
slicesParser_amplitudes(path).std()

transient1    0.197490
transient2    0.185651
transient3    0.189064
transient4    0.185952
transient5    0.201444
dtype: float64